# Preprocesamiento y Escalamiento de Datos


In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer

## Concepto de Preprocesamiento
El preprocesamiento de datos es esencial para preparar los datos en bruto antes del análisis y modelado.

In [17]:
datos_brutos = {
    'ID_Cliente': [1, 2, 3, 4, 5],
    'Edad': [28, np.nan, 45, 32, 29],
    'Ingresos': [50000, 65000, 72000, np.nan, 58000],
    'Ciudad': ['Madrid', 'Barcelona', 'Valencia', 'Madrid', 'Sevilla'],
    'Compras_Anuales': [5, 3, np.nan, 4, 2]
}
df_bruto = pd.DataFrame(datos_brutos)
df_bruto

,ID_Cliente,Edad,Ingresos,Ciudad,Compras_Anuales
0,1,28.0,50000.0,Madrid,5.0
1,2,NaN,65000.0,Barcelona,3.0
2,3,45.0,72000.0,Valencia,NaN
3,4,32.0,NaN,Madrid,4.0
4,5,29.0,58000.0,Sevilla,2.0


## Identificación de problemas en datos brutos
Valores faltantes y duplicados afectan la calidad de los datos.

In [18]:
datos_problemas = {
    'Producto': ['A', 'B', 'A', 'C', 'B', 'A'],
    'Precio': [100, 150, 100, np.nan, 150, 120],
    'Cantidad': [1, 2, 1, 3, 2, 1]
}
df_problemas = pd.DataFrame(datos_problemas)
df_problemas.isnull().sum(), df_problemas.duplicated().sum()

(Producto    0
 Precio      1
 Cantidad    0
 dtype: int64,
 np.int64(2))

## Limpieza básica de datos
Eliminación de nulos y duplicados.

In [19]:
df_limpio_nulos = df_problemas.dropna()
df_duplicados = pd.DataFrame({'ID': [1, 2, 1, 3, 2, 1], 'Valor': [10, 20, 10, 30, 20, 10]})
df_sin_duplicados = df_duplicados.drop_duplicates()
df_limpio_nulos, df_sin_duplicados

(  Producto  Precio  Cantidad
 0        A   100.0         1
 1        B   150.0         2
 2        A   100.0         1
 4        B   150.0         2
 5        A   120.0         1,
    ID  Valor
 0   1     10
 1   2     20
 3   3     30)

## Normalización manual de datos
Escalar variables a un rango específico.

In [20]:
data_escalas = {'Caracteristica_A': [10, 20, 30, 40, 50], 'Caracteristica_B': [100, 200, 300, 400, 500]}
df_escalas = pd.DataFrame(data_escalas)
min_val_A = df_escalas['Caracteristica_A'].min()
max_val_A = df_escalas['Caracteristica_A'].max()
df_escalas['Caracteristica_A_Normalizada'] = (df_escalas['Caracteristica_A'] - min_val_A) / (max_val_A - min_val_A)
df_escalas

,Caracteristica_A,Caracteristica_B,Caracteristica_A_Normalizada
0,10,100,0.00
1,20,200,0.25
2,30,300,0.50
3,40,400,0.75
4,50,500,1.00


## Transformación logarítmica y Label Encoding
Mejorar la distribución y codificar variables categóricas con orden.

In [21]:
df_transform = pd.DataFrame({'Valor_Positivo': [1, 10, 100, 1000, 10000]})
df_transform['Valor_Positivo_Log'] = np.log(df_transform['Valor_Positivo'])
df_categorias = pd.DataFrame({'Tamaño': ['Pequeño', 'Mediano', 'Grande', 'Pequeño', 'Mediano'], 'Nivel_Urgencia': ['Baja', 'Media', 'Alta', 'Baja', 'Media']})
le = LabelEncoder()
df_categorias['Tamaño_Encoded'] = le.fit_transform(df_categorias['Tamaño'])
df_transform, df_categorias

(   Valor_Positivo  Valor_Positivo_Log
 0               1            0.000000
 1              10            2.302585
 2             100            4.605170
 3            1000            6.907755
 4           10000            9.210340,
     Tamaño Nivel_Urgencia  Tamaño_Encoded
 0  Pequeño           Baja               2
 1  Mediano          Media               1
 2   Grande           Alta               0
 3  Pequeño           Baja               2
 4  Mediano          Media               1)

## One-Hot Encoding con scikit-learn
Codificación de variables categóricas sin orden natural.

In [ ]:
df_color = pd.DataFrame({'Color': ['Rojo', 'Azul', 'Verde', 'Rojo', 'Azul'], 'Material': ['Madera', 'Metal', 'Plástico', 'Madera', 'Metal']})
ct = ColumnTransformer([('encoder', OneHotEncoder(), ['Color'])], remainder='passthrough')
color_encoded_array = ct.fit_transform(df_color)
feature_names = ct.named_transformers_['encoder'].get_feature_names_out(['Color'])
df_encoded = pd.DataFrame(color_encoded_array, columns=list(feature_names) + ['Material'])
df_encoded

,Color_Azul,Color_Rojo,Color_Verde,Material
0,0.0,1.0,0.0,Madera
1,1.0,0.0,0.0,Metal
2,0.0,0.0,1.0,Plástico
3,0.0,1.0,0.0,Madera
4,1.0,0.0,0.0,Metal


## Creación de Variables Dummy con pandas
Variables binarias para categorías usando pandas.get_dummies().

In [23]:
df_ciudad = pd.DataFrame({'Ciudad': ['Nueva York', 'Londres', 'París', 'Nueva York', 'Berlín'], 'Poblacion': [8.4, 8.9, 2.1, 8.4, 3.7]})
df_dummy = pd.get_dummies(df_ciudad, columns=['Ciudad'], prefix='Ciudad')
df_dummy

,Poblacion,Ciudad_Berlín,Ciudad_Londres,Ciudad_Nueva York,Ciudad_París
0,8.4,False,False,True,False
1,8.9,False,True,False,False
2,2.1,False,False,False,True
3,8.4,False,False,True,False
4,3.7,True,False,False,False


## Escalamiento y cálculo de Distancia Manhattan

La distancia Manhattan mide la suma de las diferencias absolutas entre las características de dos puntos.  
Es muy utilizada en algoritmos de machine learning que dependen de cálculos de distancia, como k-NN y clustering.

**Importancia del escalamiento:**  
Si las variables tienen escalas muy diferentes (por ejemplo, 'Gasto_Marketing' en miles y 'Num_Empleados' en unidades),  
la variable con mayor escala dominará el cálculo de la distancia y puede sesgar los resultados del modelo.  
Por eso, es fundamental escalar las variables antes de aplicar estos métodos.

A continuación, se calcula la distancia Manhattan entre dos empresas con diferentes gastos y

In [24]:
datos_desescalados = pd.DataFrame({'Gasto_Marketing': [10000, 15000, 12000, 18000, 16000], 'Num_Empleados': [10, 15, 12, 18, 16]})
punto_A = datos_desescalados.iloc[0]
punto_B = datos_desescalados.iloc[1]
distancia_manhattan = abs(punto_B['Gasto_Marketing'] - punto_A['Gasto_Marketing']) + abs(punto_B['Num_Empleados'] - punto_A['Num_Empleados'])
distancia_manhattan # np.int64(5005)

# Esto significa que, sumando las diferencias absolutas de 'Gasto_Marketing' y 'Num_Empleados' entre ambas empresas, la distancia total es 5005 unidades.

# Interpretación:
# La mayor parte de la distancia proviene de la diferencia en 'Gasto_Marketing' (5000), mientras que la diferencia en 'Num_Empleados' (5) aporta muy poco.
# Esto muestra que, si no se escalan las variables, la característica con mayor escala domina el cálculo de la distancia y puede sesgar el análisis.

np.int64(5005)

In [ ]:
# Escalar los datos
scaler = MinMaxScaler()
datos_escalados = scaler.fit_transform(datos_desescalados)
df_escalado = pd.DataFrame(datos_escalados, columns=datos_desescalados.columns)

# Calcular la distancia Manhattan con los datos escalados
punto_A_esc = df_escalado.iloc[0]
punto_B_esc = df_escalado.iloc[1]
distancia_manhattan_esc = abs(punto_B_esc['Gasto_Marketing'] - punto_A_esc['Gasto_Marketing']) + abs(punto_B_esc['Num_Empleados'] - punto_A_esc['Num_Empleados'])
distancia_manhattan_esc # np.float64(1.25)

# Ahora la distancia Manhattan refleja el aporte de ambas variables en la misma escala.

# Interpretación:
# Después de escalar, ambas variables ('Gasto_Marketing' y 'Num_Empleados') tienen el mismo rango (0 a 1), por lo que su aporte a la distancia es proporcional y no está dominado por ninguna de ellas.
# Esto permite comparar empresas considerando ambos factores de manera justa y evita sesgos por diferencias de escala.

np.float64(1.25)

## Distancia Euclidiana y Escalador Min-Max
Cálculo de distancia y normalización con MinMaxScaler.

In [26]:
distancia_euclidiana = np.sqrt((punto_B['Gasto_Marketing'] - punto_A['Gasto_Marketing'])**2 + (punto_B['Num_Empleados'] - punto_A['Num_Empleados'])**2)
scaler_minmax = MinMaxScaler()
datos_escalados_minmax_array = scaler_minmax.fit_transform(datos_desescalados)
df_escalado_minmax = pd.DataFrame(datos_escalados_minmax_array, columns=datos_desescalados.columns)
distancia_euclidiana, df_escalado_minmax

(np.float64(5000.002499999375),
    Gasto_Marketing  Num_Empleados
 0            0.000          0.000
 1            0.625          0.625
 2            0.250          0.250
 3            1.000          1.000
 4            0.750          0.750)

## Escalador Estándar
Estandarización de datos con media 0 y desviación estándar 1.

In [27]:
scaler_standard = StandardScaler()
datos_escalados_standard_array = scaler_standard.fit_transform(datos_desescalados)
df_escalado_standard = pd.DataFrame(datos_escalados_standard_array, columns=datos_desescalados.columns)
df_escalado_standard

,Gasto_Marketing,Num_Empleados
0,-1.470294,-1.470294
1,0.280056,0.280056
2,-0.770154,-0.770154
3,1.330266,1.330266
4,0.630126,0.630126


## Comparación de MinMaxScaler y StandardScaler
Efecto de los outliers en ambos métodos de escalamiento.

In [28]:
data_outlier = pd.DataFrame({'Caracteristica_Num': [1, 2, 3, 4, 100], 'Otra_Caracteristica': [10, 20, 30, 40, 50]})
scaler_minmax_comp = MinMaxScaler()
df_minmax_comp = pd.DataFrame(scaler_minmax_comp.fit_transform(data_outlier), columns=data_outlier.columns)
scaler_standard_comp = StandardScaler()
df_standard_comp = pd.DataFrame(scaler_standard_comp.fit_transform(data_outlier), columns=data_outlier.columns)
df_minmax_comp, df_standard_comp

(   Caracteristica_Num  Otra_Caracteristica
 0            0.000000                 0.00
 1            0.010101                 0.25
 2            0.020202                 0.50
 3            0.030303                 0.75
 4            1.000000                 1.00,
    Caracteristica_Num  Otra_Caracteristica
 0           -0.538285            -1.414214
 1           -0.512652            -0.707107
 2           -0.487019             0.000000
 3           -0.461387             0.707107
 4            1.999343             1.414214)

## Actividad práctica guiada de escalamiento
Aplicación de MinMaxScaler y StandardScaler a un dataset de ejemplo.

In [29]:
data_actividad = {
    'Caracteristica_A': [10, 20, 30, 40, 50],
    'Caracteristica_B': [100, 200, 300, 400, 500],
    'Caracteristica_C': [5, 3, 8, 2, 7]
}
df_actividad = pd.DataFrame(data_actividad)
minmax_scaler_act = MinMaxScaler()
df_actividad_minmax = pd.DataFrame(minmax_scaler_act.fit_transform(df_actividad), columns=df_actividad.columns)
standard_scaler_act = StandardScaler()
df_actividad_standard = pd.DataFrame(standard_scaler_act.fit_transform(df_actividad), columns=df_actividad.columns)
df_actividad_minmax, df_actividad_standard

(   Caracteristica_A  Caracteristica_B  Caracteristica_C
 0              0.00              0.00          0.500000
 1              0.25              0.25          0.166667
 2              0.50              0.50          1.000000
 3              0.75              0.75          0.000000
 4              1.00              1.00          0.833333,
    Caracteristica_A  Caracteristica_B  Caracteristica_C
 0         -1.414214         -1.414214          0.000000
 1         -0.707107         -0.707107         -0.877058
 2          0.000000          0.000000          1.315587
 3          0.707107          0.707107         -1.315587
 4          1.414214          1.414214          0.877058)